# Neighborhood graphs

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [1]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

------

## In a Grid graph

In [12]:
#% js
my $g = Graph::Grid.new(4, 6);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [13]:
my $g2 = $g.neighborhood-graph(['2_1', '2_3'], d => 2)

Graph(vertexes => 18, edges => 25, directed => False)

In [14]:
#% js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## In a Petersen graph

In [8]:
#% js
my $g = Graph::Petersen.new;

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [9]:
my $g2 = $g.neighborhood-graph(["0", "1"], d => 1)

Graph(vertexes => 7, edges => 8, directed => False)

In [10]:
#%js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

-----

## Neat examples

### Pick out random neighborhoods from a grid

In [45]:
my $g = Graph::Grid.new(30, 60);

my $g2 = $g.neighborhood-graph($g.vertex-list.pick(50), d => 1)

Graph(vertexes => 237, edges => 210, directed => False)

In [48]:
#%js
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });

$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :%vertex-coordinates,
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 400,
        edge-thickness => 3,
        vertex-size => 0, 
        force => {charge => {strength => -100, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

### Subtract random neighborhoods from a grid

In [104]:
my $g = Graph::Grid.new(20, 30);

my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(25), d => 1).vertex-list;

my $g2 = $g.subgraph($vset.keys)

Graph(vertexes => 575, edges => 1042, directed => False)

In [105]:
#%js
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });

$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :%vertex-coordinates,
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 400,
        edge-thickness => 1,
        vertex-size => 0,
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )